# BBC Text Representations - Dense Methods

**Roll Number:** SE22UARI195

**Tasks:**
1. Word2Vec Skip-gram with Negative Sampling (NS)
2. Word2Vec CBOW with Negative Sampling (NS)
3. Word2Vec Skip-gram with Hierarchical Softmax (HS)
4. Word2Vec CBOW with Hierarchical Softmax (HS)
5. GloVe (pretrained 100d)
6. TF-IDF weighted pooling for document vectors
7. Compare training speed and quality

---

## 1. Setup & Load Preprocessed Data

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import pickle
import time
from pathlib import Path
from collections import Counter

# Gensim for Word2Vec
from gensim.models import Word2Vec

# Sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Progress bar
from tqdm.notebook import tqdm

print("✅ Imports successful!")

In [ ]:
# Configuration
ROLL = "SE22UARI195"
CACHE_DIR = Path("../cache")
MODELS_DIR = Path("../models")
DATA_DIR = Path("../data")

print(f"Roll Number: {ROLL}")
print(f"Cache Directory: {CACHE_DIR}")
print(f"Models Directory: {MODELS_DIR}")
print(f"Data Directory: {DATA_DIR}")

In [ ]:
# Load preprocessed data
print("📂 Loading preprocessed data...\n")

with open(CACHE_DIR / 'train_processed.pkl', 'rb') as f:
    train_df = pickle.load(f)
print(f"✅ TRAIN: {len(train_df)} documents")

with open(CACHE_DIR / 'dev_processed.pkl', 'rb') as f:
    dev_df = pickle.load(f)
print(f"✅ DEV: {len(dev_df)} documents")

with open(CACHE_DIR / 'test_processed.pkl', 'rb') as f:
    test_df = pickle.load(f)
print(f"✅ TEST: {len(test_df)} documents")

with open(CACHE_DIR / 'vocab_counter.pkl', 'rb') as f:
    vocab_counter = pickle.load(f)
print(f"✅ Vocabulary: {len(vocab_counter):,} unique tokens")

In [ ]:
# Load TF-IDF vectorizer (needed for weighted pooling)
with open(MODELS_DIR / 'tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

print(f"✅ TF-IDF vectorizer loaded")
print(f"   Vocabulary size: {len(tfidf_vectorizer.vocabulary_):,}")

In [ ]:
# Prepare tokenized sentences for Word2Vec
train_sentences = train_df['tokens'].tolist()
dev_sentences = dev_df['tokens'].tolist()
test_sentences = test_df['tokens'].tolist()

print("\n📊 Tokenized sentences:")
print(f"  TRAIN: {len(train_sentences)} documents")
print(f"  DEV: {len(dev_sentences)} documents")
print(f"  TEST: {len(test_sentences)} documents")
print(f"\n  Sample: {train_sentences[0][:15]}...")

## 2. Helper Functions

In [ ]:
def tfidf_weighted_pooling(tokens_list, word_vectors, tfidf_vec, tfidf_vocab):
    """
    Convert list of token lists to document vectors using TF-IDF weighted mean.
    
    Args:
        tokens_list: List of token lists
        word_vectors: Word embedding model (Word2Vec or dict of embeddings)
        tfidf_vec: Fitted TfidfVectorizer
        tfidf_vocab: TF-IDF vocabulary dict
    
    Returns:
        numpy array of shape (n_docs, embedding_dim)
    """
    # Get embedding dimension
    if hasattr(word_vectors, 'wv'):
        # Word2Vec model
        vector_size = word_vectors.wv.vector_size
        vocab = word_vectors.wv
    else:
        # Dictionary of embeddings (GloVe)
        vector_size = len(next(iter(word_vectors.values())))
        vocab = word_vectors
    
    doc_vectors = []
    
    for tokens in tqdm(tokens_list, desc="Pooling"):
        if len(tokens) == 0:
            # Empty document - use zero vector
            doc_vectors.append(np.zeros(vector_size))
            continue
        
        # Get TF-IDF weights for this document
        text = ' '.join(tokens)
        tfidf_vec_doc = tfidf_vec.transform([text]).toarray()[0]
        
        # Accumulate weighted vectors
        weighted_sum = np.zeros(vector_size)
        total_weight = 0.0
        
        for token in tokens:
            # Check if token is in both embeddings and TF-IDF vocab
            if token in tfidf_vocab:
                tfidf_idx = tfidf_vocab[token]
                tfidf_weight = tfidf_vec_doc[tfidf_idx]
                
                # Get word vector
                if hasattr(vocab, '__contains__'):
                    if token in vocab:
                        if hasattr(vocab, 'get_vector'):
                            word_vec = vocab.get_vector(token)
                        else:
                            word_vec = vocab[token]
                        weighted_sum += tfidf_weight * word_vec
                        total_weight += tfidf_weight
                else:
                    if token in vocab:
                        word_vec = vocab[token]
                        weighted_sum += tfidf_weight * word_vec
                        total_weight += tfidf_weight
        
        # Average by total weight
        if total_weight > 0:
            doc_vectors.append(weighted_sum / total_weight)
        else:
            # No valid tokens - use zero vector
            doc_vectors.append(np.zeros(vector_size))
    
    return np.array(doc_vectors)

print("✅ Helper functions defined!")

## 3. Word2Vec - Skip-gram with Negative Sampling (NS)

In [ ]:
print("\n🔧 Training Word2Vec Skip-gram with Negative Sampling...\n")

# Count total tokens for speed calculation
total_tokens_train = sum(len(sent) for sent in train_sentences)
print(f"Total training tokens: {total_tokens_train:,}")

# Train Word2Vec Skip-gram with NS
start_time = time.time()

w2v_sg_ns = Word2Vec(
    sentences=train_sentences,
    sg=1,  # Skip-gram
    vector_size=100,
    window=5,
    min_count=3,
    negative=5,  # Negative sampling with k=5
    hs=0,  # No hierarchical softmax
    epochs=10,
    workers=4,
    seed=42
)

train_time_sg_ns = time.time() - start_time
tokens_per_sec_sg_ns = total_tokens_train / train_time_sg_ns

print(f"✅ Training complete in {train_time_sg_ns:.2f}s")
print(f"   Tokens/sec: {tokens_per_sec_sg_ns:,.0f}")
print(f"   Vocabulary size: {len(w2v_sg_ns.wv):,}")
print(f"   Vector size: {w2v_sg_ns.wv.vector_size}")

In [ ]:
# Save model
w2v_sg_ns.save(str(MODELS_DIR / 'w2v_sg_ns.model'))
print("💾 Model saved!")

In [ ]:
# Test nearest neighbors
print("\n🔍 Nearest neighbors (Skip-gram NS):")
test_words = ['government', 'film', 'music', 'technology', 'economy']

for word in test_words:
    if word in w2v_sg_ns.wv:
        similar = w2v_sg_ns.wv.most_similar(word, topn=5)
        print(f"\n  {word}:")
        for sim_word, score in similar:
            print(f"    {sim_word:15s} : {score:.4f}")
    else:
        print(f"\n  {word}: NOT IN VOCABULARY")

In [ ]:
# Create document vectors using TF-IDF weighted pooling
print("\n📊 Creating document vectors with TF-IDF weighted pooling...")

X_train_w2v_sg_ns = tfidf_weighted_pooling(
    train_sentences, w2v_sg_ns, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

X_dev_w2v_sg_ns = tfidf_weighted_pooling(
    dev_sentences, w2v_sg_ns, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

X_test_w2v_sg_ns = tfidf_weighted_pooling(
    test_sentences, w2v_sg_ns, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

print(f"\n✅ Document vectors created:")
print(f"   TRAIN: {X_train_w2v_sg_ns.shape}")
print(f"   DEV: {X_dev_w2v_sg_ns.shape}")
print(f"   TEST: {X_test_w2v_sg_ns.shape}")

In [ ]:
# Save document vectors
np.save(MODELS_DIR / 'X_train_w2v_sg_ns.npy', X_train_w2v_sg_ns)
np.save(MODELS_DIR / 'X_dev_w2v_sg_ns.npy', X_dev_w2v_sg_ns)
np.save(MODELS_DIR / 'X_test_w2v_sg_ns.npy', X_test_w2v_sg_ns)
print("\n💾 Document vectors saved!")

## 4. Word2Vec - CBOW with Negative Sampling (NS)

In [ ]:
print("\n🔧 Training Word2Vec CBOW with Negative Sampling...\n")

start_time = time.time()

w2v_cbow_ns = Word2Vec(
    sentences=train_sentences,
    sg=0,  # CBOW
    vector_size=100,
    window=5,
    min_count=3,
    negative=5,
    hs=0,
    epochs=10,
    workers=4,
    seed=42
)

train_time_cbow_ns = time.time() - start_time
tokens_per_sec_cbow_ns = total_tokens_train / train_time_cbow_ns

print(f"✅ Training complete in {train_time_cbow_ns:.2f}s")
print(f"   Tokens/sec: {tokens_per_sec_cbow_ns:,.0f}")
print(f"   Vocabulary size: {len(w2v_cbow_ns.wv):,}")

In [ ]:
# Save model
w2v_cbow_ns.save(str(MODELS_DIR / 'w2v_cbow_ns.model'))
print("💾 Model saved!")

In [ ]:
# Test nearest neighbors
print("\n🔍 Nearest neighbors (CBOW NS):")

for word in test_words:
    if word in w2v_cbow_ns.wv:
        similar = w2v_cbow_ns.wv.most_similar(word, topn=5)
        print(f"\n  {word}:")
        for sim_word, score in similar:
            print(f"    {sim_word:15s} : {score:.4f}")
    else:
        print(f"\n  {word}: NOT IN VOCABULARY")

In [ ]:
# Create document vectors
print("\n📊 Creating document vectors with TF-IDF weighted pooling...")

X_train_w2v_cbow_ns = tfidf_weighted_pooling(
    train_sentences, w2v_cbow_ns, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

X_dev_w2v_cbow_ns = tfidf_weighted_pooling(
    dev_sentences, w2v_cbow_ns, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

X_test_w2v_cbow_ns = tfidf_weighted_pooling(
    test_sentences, w2v_cbow_ns, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

print(f"\n✅ Document vectors created:")
print(f"   TRAIN: {X_train_w2v_cbow_ns.shape}")
print(f"   DEV: {X_dev_w2v_cbow_ns.shape}")
print(f"   TEST: {X_test_w2v_cbow_ns.shape}")

In [ ]:
# Save document vectors
np.save(MODELS_DIR / 'X_train_w2v_cbow_ns.npy', X_train_w2v_cbow_ns)
np.save(MODELS_DIR / 'X_dev_w2v_cbow_ns.npy', X_dev_w2v_cbow_ns)
np.save(MODELS_DIR / 'X_test_w2v_cbow_ns.npy', X_test_w2v_cbow_ns)
print("\n💾 Document vectors saved!")

## 5. Word2Vec - Skip-gram with Hierarchical Softmax (HS)

In [ ]:
print("\n🔧 Training Word2Vec Skip-gram with Hierarchical Softmax...\n")

start_time = time.time()

w2v_sg_hs = Word2Vec(
    sentences=train_sentences,
    sg=1,  # Skip-gram
    vector_size=100,
    window=5,
    min_count=3,
    negative=0,  # No negative sampling
    hs=1,  # Hierarchical softmax
    epochs=10,
    workers=4,
    seed=42
)

train_time_sg_hs = time.time() - start_time
tokens_per_sec_sg_hs = total_tokens_train / train_time_sg_hs

print(f"✅ Training complete in {train_time_sg_hs:.2f}s")
print(f"   Tokens/sec: {tokens_per_sec_sg_hs:,.0f}")
print(f"   Vocabulary size: {len(w2v_sg_hs.wv):,}")

In [ ]:
# Save model
w2v_sg_hs.save(str(MODELS_DIR / 'w2v_sg_hs.model'))
print("💾 Model saved!")

In [ ]:
# Test nearest neighbors
print("\n🔍 Nearest neighbors (Skip-gram HS):")

for word in test_words:
    if word in w2v_sg_hs.wv:
        similar = w2v_sg_hs.wv.most_similar(word, topn=5)
        print(f"\n  {word}:")
        for sim_word, score in similar:
            print(f"    {sim_word:15s} : {score:.4f}")
    else:
        print(f"\n  {word}: NOT IN VOCABULARY")

In [ ]:
# Create document vectors
print("\n📊 Creating document vectors with TF-IDF weighted pooling...")

X_train_w2v_sg_hs = tfidf_weighted_pooling(
    train_sentences, w2v_sg_hs, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

X_dev_w2v_sg_hs = tfidf_weighted_pooling(
    dev_sentences, w2v_sg_hs, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

X_test_w2v_sg_hs = tfidf_weighted_pooling(
    test_sentences, w2v_sg_hs, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

print(f"\n✅ Document vectors created:")
print(f"   TRAIN: {X_train_w2v_sg_hs.shape}")
print(f"   DEV: {X_dev_w2v_sg_hs.shape}")
print(f"   TEST: {X_test_w2v_sg_hs.shape}")

In [ ]:
# Save document vectors
np.save(MODELS_DIR / 'X_train_w2v_sg_hs.npy', X_train_w2v_sg_hs)
np.save(MODELS_DIR / 'X_dev_w2v_sg_hs.npy', X_dev_w2v_sg_hs)
np.save(MODELS_DIR / 'X_test_w2v_sg_hs.npy', X_test_w2v_sg_hs)
print("\n💾 Document vectors saved!")

## 6. Word2Vec - CBOW with Hierarchical Softmax (HS)

In [ ]:
print("\n🔧 Training Word2Vec CBOW with Hierarchical Softmax...\n")

start_time = time.time()

w2v_cbow_hs = Word2Vec(
    sentences=train_sentences,
    sg=0,  # CBOW
    vector_size=100,
    window=5,
    min_count=3,
    negative=0,
    hs=1,  # Hierarchical softmax
    epochs=10,
    workers=4,
    seed=42
)

train_time_cbow_hs = time.time() - start_time
tokens_per_sec_cbow_hs = total_tokens_train / train_time_cbow_hs

print(f"✅ Training complete in {train_time_cbow_hs:.2f}s")
print(f"   Tokens/sec: {tokens_per_sec_cbow_hs:,.0f}")
print(f"   Vocabulary size: {len(w2v_cbow_hs.wv):,}")

In [ ]:
# Save model
w2v_cbow_hs.save(str(MODELS_DIR / 'w2v_cbow_hs.model'))
print("💾 Model saved!")

In [ ]:
# Test nearest neighbors
print("\n🔍 Nearest neighbors (CBOW HS):")

for word in test_words:
    if word in w2v_cbow_hs.wv:
        similar = w2v_cbow_hs.wv.most_similar(word, topn=5)
        print(f"\n  {word}:")
        for sim_word, score in similar:
            print(f"    {sim_word:15s} : {score:.4f}")
    else:
        print(f"\n  {word}: NOT IN VOCABULARY")

In [ ]:
# Create document vectors
print("\n📊 Creating document vectors with TF-IDF weighted pooling...")

X_train_w2v_cbow_hs = tfidf_weighted_pooling(
    train_sentences, w2v_cbow_hs, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

X_dev_w2v_cbow_hs = tfidf_weighted_pooling(
    dev_sentences, w2v_cbow_hs, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

X_test_w2v_cbow_hs = tfidf_weighted_pooling(
    test_sentences, w2v_cbow_hs, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

print(f"\n✅ Document vectors created:")
print(f"   TRAIN: {X_train_w2v_cbow_hs.shape}")
print(f"   DEV: {X_dev_w2v_cbow_hs.shape}")
print(f"   TEST: {X_test_w2v_cbow_hs.shape}")

In [ ]:
# Save document vectors
np.save(MODELS_DIR / 'X_train_w2v_cbow_hs.npy', X_train_w2v_cbow_hs)
np.save(MODELS_DIR / 'X_dev_w2v_cbow_hs.npy', X_dev_w2v_cbow_hs)
np.save(MODELS_DIR / 'X_test_w2v_cbow_hs.npy', X_test_w2v_cbow_hs)
print("\n💾 Document vectors saved!")

## 7. GloVe Embeddings (Pretrained)

In [ ]:
print("\n📥 Loading GloVe embeddings...\n")

glove_path = DATA_DIR / 'glove.6B.100d.txt'

if not glove_path.exists():
    print("❌ ERROR: GloVe file not found!")
    print(f"   Expected location: {glove_path}")
    print("\n   Please download from: http://nlp.stanford.edu/data/glove.6B.zip")
    print("   Extract glove.6B.100d.txt to the data/ folder")
    raise FileNotFoundError(f"GloVe embeddings not found at {glove_path}")

# Load GloVe embeddings into dictionary
glove_embeddings = {}

with open(glove_path, 'r', encoding='utf-8') as f:
    for line in tqdm(f, desc="Loading GloVe", total=400000):
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector

print(f"\n✅ Loaded {len(glove_embeddings):,} word vectors")
print(f"   Vector dimension: {len(next(iter(glove_embeddings.values())))}")

In [ ]:
# Test nearest neighbors for GloVe
print("\n🔍 Testing GloVe embeddings:")

def get_most_similar_glove(word, embeddings, topn=5):
    """Find most similar words using cosine similarity."""
    if word not in embeddings:
        return None
    
    word_vec = embeddings[word]
    similarities = {}
    
    # Calculate similarities with a subset for speed
    for other_word, other_vec in list(embeddings.items())[:50000]:
        if other_word == word:
            continue
        sim = np.dot(word_vec, other_vec) / (np.linalg.norm(word_vec) * np.linalg.norm(other_vec))
        similarities[other_word] = sim
    
    return sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:topn]

for word in test_words:
    if word in glove_embeddings:
        similar = get_most_similar_glove(word, glove_embeddings, topn=5)
        print(f"\n  {word}:")
        if similar:
            for sim_word, score in similar:
                print(f"    {sim_word:15s} : {score:.4f}")
    else:
        print(f"\n  {word}: NOT IN GLOVE VOCABULARY")

In [ ]:
# Create document vectors using GloVe
print("\n📊 Creating document vectors with GloVe + TF-IDF pooling...")

X_train_glove = tfidf_weighted_pooling(
    train_sentences, glove_embeddings, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

X_dev_glove = tfidf_weighted_pooling(
    dev_sentences, glove_embeddings, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

X_test_glove = tfidf_weighted_pooling(
    test_sentences, glove_embeddings, tfidf_vectorizer, tfidf_vectorizer.vocabulary_
)

print(f"\n✅ Document vectors created:")
print(f"   TRAIN: {X_train_glove.shape}")
print(f"   DEV: {X_dev_glove.shape}")
print(f"   TEST: {X_test_glove.shape}")

In [ ]:
# Save GloVe document vectors
np.save(MODELS_DIR / 'X_train_glove.npy', X_train_glove)
np.save(MODELS_DIR / 'X_dev_glove.npy', X_dev_glove)
np.save(MODELS_DIR / 'X_test_glove.npy', X_test_glove)
print("\n💾 GloVe document vectors saved!")

## 8. Calculate Health Metrics for Dense Methods

In [ ]:
print("\n📊 Calculating health metrics for dense representations...\n")

# Calculate OOV rates
def calculate_oov_rate(tokens_list, vocab):
    """Calculate out-of-vocabulary rate."""
    total_tokens = 0
    oov_tokens = 0
    
    for tokens in tokens_list:
        for token in tokens:
            total_tokens += 1
            if token not in vocab:
                oov_tokens += 1
    
    return (oov_tokens / total_tokens * 100) if total_tokens > 0 else 0.0

# Calculate coverage
def calculate_coverage(tokens_list, vocab, k):
    """Calculate top-k coverage."""
    token_counts = Counter()
    for tokens in tokens_list:
        token_counts.update(tokens)
    
    top_k_tokens = [token for token, _ in token_counts.most_common(k)]
    covered = sum(1 for token in top_k_tokens if token in vocab)
    
    return (covered / k * 100) if k > 0 else 0.0

print("✅ Helper functions defined!")

In [ ]:
# Word2Vec Skip-gram NS metrics
oov_sg_ns = calculate_oov_rate(test_sentences, w2v_sg_ns.wv)
cov100_sg_ns = calculate_coverage(test_sentences, w2v_sg_ns.wv, 100)
cov500_sg_ns = calculate_coverage(test_sentences, w2v_sg_ns.wv, 500)

print("Word2Vec Skip-gram NS:")
print(f"  Vocabulary size: {len(w2v_sg_ns.wv):,}")
print(f"  OOV rate (TEST): {oov_sg_ns:.2f}%")
print(f"  Top-100 coverage: {cov100_sg_ns:.2f}%")
print(f"  Top-500 coverage: {cov500_sg_ns:.2f}%")
print(f"  Training time: {train_time_sg_ns:.2f}s")
print(f"  Tokens/sec: {tokens_per_sec_sg_ns:,.0f}")

In [ ]:
# Word2Vec CBOW NS metrics
oov_cbow_ns = calculate_oov_rate(test_sentences, w2v_cbow_ns.wv)
cov100_cbow_ns = calculate_coverage(test_sentences, w2v_cbow_ns.wv, 100)
cov500_cbow_ns = calculate_coverage(test_sentences, w2v_cbow_ns.wv, 500)

print("\nWord2Vec CBOW NS:")
print(f"  Vocabulary size: {len(w2v_cbow_ns.wv):,}")
print(f"  OOV rate (TEST): {oov_cbow_ns:.2f}%")
print(f"  Top-100 coverage: {cov100_cbow_ns:.2f}%")
print(f"  Top-500 coverage: {cov500_cbow_ns:.2f}%")
print(f"  Training time: {train_time_cbow_ns:.2f}s")
print(f"  Tokens/sec: {tokens_per_sec_cbow_ns:,.0f}")

In [ ]:
# Word2Vec Skip-gram HS metrics
oov_sg_hs = calculate_oov_rate(test_sentences, w2v_sg_hs.wv)
cov100_sg_hs = calculate_coverage(test_sentences, w2v_sg_hs.wv, 100)
cov500_sg_hs = calculate_coverage(test_sentences, w2v_sg_hs.wv, 500)

print("\nWord2Vec Skip-gram HS:")
print(f"  Vocabulary size: {len(w2v_sg_hs.wv):,}")
print(f"  OOV rate (TEST): {oov_sg_hs:.2f}%")
print(f"  Top-100 coverage: {cov100_sg_hs:.2f}%")
print(f"  Top-500 coverage: {cov500_sg_hs:.2f}%")
print(f"  Training time: {train_time_sg_hs:.2f}s")
print(f"  Tokens/sec: {tokens_per_sec_sg_hs:,.0f}")

In [ ]:
# Word2Vec CBOW HS metrics
oov_cbow_hs = calculate_oov_rate(test_sentences, w2v_cbow_hs.wv)
cov100_cbow_hs = calculate_coverage(test_sentences, w2v_cbow_hs.wv, 100)
cov500_cbow_hs = calculate_coverage(test_sentences, w2v_cbow_hs.wv, 500)

print("\nWord2Vec CBOW HS:")
print(f"  Vocabulary size: {len(w2v_cbow_hs.wv):,}")
print(f"  OOV rate (TEST): {oov_cbow_hs:.2f}%")
print(f"  Top-100 coverage: {cov100_cbow_hs:.2f}%")
print(f"  Top-500 coverage: {cov500_cbow_hs:.2f}%")
print(f"  Training time: {train_time_cbow_hs:.2f}s")
print(f"  Tokens/sec: {tokens_per_sec_cbow_hs:,.0f}")

In [ ]:
# GloVe metrics
oov_glove = calculate_oov_rate(test_sentences, glove_embeddings)
cov100_glove = calculate_coverage(test_sentences, glove_embeddings, 100)
cov500_glove = calculate_coverage(test_sentences, glove_embeddings, 500)

print("\nGloVe (pretrained):")
print(f"  Vocabulary size: {len(glove_embeddings):,}")
print(f"  OOV rate (TEST): {oov_glove:.2f}%")
print(f"  Top-100 coverage: {cov100_glove:.2f}%")
print(f"  Top-500 coverage: {cov500_glove:.2f}%")
print(f"  Training time: 0.00s (pretrained)")
print(f"  Tokens/sec: N/A (pretrained)")

## 9. Save Results Summary

In [ ]:
import json

# Compile dense methods results
dense_results = {
    'w2v_sg_ns': {
        'vocab_size': len(w2v_sg_ns.wv),
        'vector_dim': w2v_sg_ns.wv.vector_size,
        'oov_rate': float(oov_sg_ns),
        'top100_coverage': float(cov100_sg_ns),
        'top500_coverage': float(cov500_sg_ns),
        'train_time_sec': float(train_time_sg_ns),
        'tokens_per_sec': float(tokens_per_sec_sg_ns),
        'train_shape': list(X_train_w2v_sg_ns.shape),
        'dev_shape': list(X_dev_w2v_sg_ns.shape),
        'test_shape': list(X_test_w2v_sg_ns.shape)
    },
    'w2v_cbow_ns': {
        'vocab_size': len(w2v_cbow_ns.wv),
        'vector_dim': w2v_cbow_ns.wv.vector_size,
        'oov_rate': float(oov_cbow_ns),
        'top100_coverage': float(cov100_cbow_ns),
        'top500_coverage': float(cov500_cbow_ns),
        'train_time_sec': float(train_time_cbow_ns),
        'tokens_per_sec': float(tokens_per_sec_cbow_ns),
        'train_shape': list(X_train_w2v_cbow_ns.shape),
        'dev_shape': list(X_dev_w2v_cbow_ns.shape),
        'test_shape': list(X_test_w2v_cbow_ns.shape)
    },
    'w2v_sg_hs': {
        'vocab_size': len(w2v_sg_hs.wv),
        'vector_dim': w2v_sg_hs.wv.vector_size,
        'oov_rate': float(oov_sg_hs),
        'top100_coverage': float(cov100_sg_hs),
        'top500_coverage': float(cov500_sg_hs),
        'train_time_sec': float(train_time_sg_hs),
        'tokens_per_sec': float(tokens_per_sec_sg_hs),
        'train_shape': list(X_train_w2v_sg_hs.shape),
        'dev_shape': list(X_dev_w2v_sg_hs.shape),
        'test_shape': list(X_test_w2v_sg_hs.shape)
    },
    'w2v_cbow_hs': {
        'vocab_size': len(w2v_cbow_hs.wv),
        'vector_dim': w2v_cbow_hs.wv.vector_size,
        'oov_rate': float(oov_cbow_hs),
        'top100_coverage': float(cov100_cbow_hs),
        'top500_coverage': float(cov500_cbow_hs),
        'train_time_sec': float(train_time_cbow_hs),
        'tokens_per_sec': float(tokens_per_sec_cbow_hs),
        'train_shape': list(X_train_w2v_cbow_hs.shape),
        'dev_shape': list(X_dev_w2v_cbow_hs.shape),
        'test_shape': list(X_test_w2v_cbow_hs.shape)
    },
    'glove': {
        'vocab_size': len(glove_embeddings),
        'vector_dim': 100,
        'oov_rate': float(oov_glove),
        'top100_coverage': float(cov100_glove),
        'top500_coverage': float(cov500_glove),
        'train_time_sec': 0.0,
        'tokens_per_sec': None,
        'train_shape': list(X_train_glove.shape),
        'dev_shape': list(X_dev_glove.shape),
        'test_shape': list(X_test_glove.shape)
    }
}

# Save results
with open(CACHE_DIR / 'dense_results.json', 'w') as f:
    json.dump(dense_results, f, indent=2)

print("\n💾 Dense methods results saved to cache/dense_results.json")

## 10. Summary Comparison

In [ ]:
print("\n" + "="*80)
print("DENSE METHODS SUMMARY")
print("="*80)

print(f"\n{'Method':<20} {'Vocab':<10} {'OOV%':<8} {'Top100%':<9} {'Top500%':<9} {'Time(s)':<10} {'Tok/s':<12}")
print("-"*80)

for method_name, metrics in dense_results.items():
    tok_s = f"{metrics['tokens_per_sec']:,.0f}" if metrics['tokens_per_sec'] else "N/A"
    print(f"{method_name:<20} {metrics['vocab_size']:<10,} {metrics['oov_rate']:<8.2f} "
          f"{metrics['top100_coverage']:<9.2f} {metrics['top500_coverage']:<9.2f} "
          f"{metrics['train_time_sec']:<10.2f} {tok_s:<12}")

print("\n" + "="*80)
print("✅ Dense methods training complete!")
print("="*80)

In [ ]:
print("\n🎉 Notebook 03: Dense Methods - COMPLETE!")
print("\nNext steps:")
print("  1. Run notebook 04: Classification")
print("  2. Train classifiers on all representations")
print("  3. Compare performance on TEST set")